In [1]:
def simple_coroutine():
    print('-> coroutine starte')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x000001A74FBD7748>

In [4]:
next(my_coro)

-> coroutine starte


In [5]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [6]:
my_coro2 = simple_coroutine()

In [7]:
my_coro2.send(123)

TypeError: can't send non-None value to a just-started generator

In [8]:
my_coro2.send(None)

-> coroutine starte


In [9]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [10]:
my_cc = simple_coro2(14)

In [11]:
from inspect import getgeneratorstate

In [12]:
getgeneratorstate(my_cc)

'GEN_CREATED'

In [13]:
next(my_cc)

-> Started: a = 14


14

In [14]:
getgeneratorstate(my_cc)

'GEN_SUSPENDED'

In [15]:
my_cc.send(28)

-> Received: b = 28


42

In [16]:
my_cc.send(99)

-> Received: c = 99


StopIteration: 

In [17]:
getgeneratorstate(my_cc)

'GEN_CLOSED'

In [30]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [19]:
coro_avg = averager()

In [20]:
next(coro_avg)

In [22]:
coro_avg.send(10)

10.0

In [23]:
coro_avg.send(30)

20.0

In [24]:
coro_avg.send(5)

15.0

In [26]:
from functools import wraps

def coroutine(func):
    """装饰器：向前执行到第一个yield表达式，预激func"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [32]:
coro_avg = averager()

In [33]:
from inspect import getgeneratorstate

In [34]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [35]:
coro_avg.send(10)

10.0

In [36]:
coro_avg.send(30)

20.0

In [37]:
coro_avg.send(5)

15.0

In [38]:
coro_avg = averager()

In [39]:
coro_avg.send(40)

40.0

In [40]:
coro_avg.send(50)

45.0

In [41]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [44]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))

    raise RuntimeError('This line should never run.')

In [45]:
exc_coro = demo_exc_handling()

In [46]:
next(exc_coro)

-> coroutine started


In [47]:
exc_coro.send(11)

-> coroutine received: 11


In [48]:
exc_coro.send(22)

-> coroutine received: 22


In [49]:
exc_coro.close()

In [50]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [51]:
exc_coro = demo_exc_handling()

In [52]:
next(exc_coro)

-> coroutine started


In [53]:
exc_coro.send(11)

-> coroutine received: 11


In [54]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [55]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [56]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [ ]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending...')

    raise RuntimeError('This line should never run.')

In [1]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [20]:
coro_avg = averager()

In [21]:
next(coro_avg)

In [22]:
coro_avg.send(10)

In [23]:
coro_avg.send(30)

In [24]:
coro_avg.send(6.5)

In [25]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [26]:
result

Result(count=3, average=15.5)

In [30]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

In [31]:
list(gen())

['A', 'B', 1, 2]

In [32]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

In [33]:
list(gen())

['A', 'B', 1, 2]

In [34]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [35]:
s = 'ABC'

In [36]:
t = tuple(range(3))

In [37]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [41]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 子生成器 
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
# 客户端代码，即调用方
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        # 重要
        group.send(None)
        report(results)
        
# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2}{:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))
        
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

if __name__ == '__main__':
    main(data)

10girls averaging 42.04kg
10girls averaging 1.43m
10girls averaging 1.43m
10girls averaging 1.43m


In [42]:
import collections

In [43]:
Event = collections.namedtuple('Event', 'time proc action')

In [45]:
def taxi_process(ident, trips, start_time=0):
    """每次改变状态时创建事件，把控制权让给仿真器"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    
    yield Event(time, ident, 'going home')

In [46]:
taxi = taxi_process(ident=13, trips=2, start_time=0)

In [47]:
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [48]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [49]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [50]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [51]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [52]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [53]:
taxi.send(_.time + 10)

StopIteration: 

In [57]:
import queue

class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time): #1
        """排定并显示事件，直到时间结束"""
        # 排定各辆出租车的第一个事件
        for _, proc in sorted(self.procs.items()): #2
            first_event = next(proc) #3
            self.events.put(first_event) #4
        
        # 这个仿真系统的主循环
        sim_time = 0 #5
        while sim_time < end_time: #6
            if self.events.empty(): #7
                print('*** end of events ***')
                break
            
            current_event = self.events.get() #8
            sim_time, proc_id, previous_action = current_event #9
            print('taxi:', proc_id, proc_id * '  ', current_event) #10
            active_proc = self.procs[proc_id] #11
            # 传入前一个动作，把结果加到sim_time上，获得下一次活动的时刻
            next_time = sim_time + compute_duration(previous_action) #12
            try:
                next_event = active_proc.send(next_time) #13
            except StopIteration:
                del self.procs[proc_id] #14
            else:
                self.events.put(next_event) #15
        
        else: #16
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))            